# Basic Q&A System

### Sentence Retrival

In [15]:
import math,numpy,json,re,nltk
import time,re,os.path,sys
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from numpy import multiply
from math import sqrt
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os.path as path
from collections import OrderedDict
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from sklearn.feature_extraction import DictVectorizer
from nltk import FreqDist, DictionaryProbDist
from operator import add

#### Load data from json file

In [2]:
t0 = time.time()
filename_ls = ['QA_test.json']
dataset = []
train_path = path.abspath('data/QA_test.json')

dataset = []
with open(train_path) as f:
    for line in f:
        dataset+=(json.loads(line))
print "Import Successful "
print "There are totally", len(dataset),'documents in this dataset'

Import Successful 
There are totally 42 documents in this dataset


#### Build modle and evaluate the result

In [3]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def my_tokenizer(doc):
    terms = set()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return list(terms)


class MostRelevantSentenceModel(object):
    def __init__(self, vectorizer, collection_matrix):
        self.vectorizer = vectorizer
        self.collection_matrix = collection_matrix
        feature_array = vectorizer.get_feature_names()
        self.features = dict()
        for index in range(len(feature_array)):
            term = feature_array[index]
            self.features[term] = index

    def predict(self, queies):
        predictions = [self.inverted_index_score(i) for i in  queies]
        return predictions

    def inverted_index_score(self, query_sent):
        """
        now we implement inverted index to handle query
        
        :param query_sent: 
        :return: 
        
        """
        query_words = my_tokenizer(query_sent)
        score = defaultdict(float)

        for w in query_words:
            try:
                col_i = self.features[w]
                inverted_ix = self.collection_matrix[:, col_i]
                for di in range(inverted_ix.shape[0]):
                    score[di] += inverted_ix[di, 0]
            except KeyError:
                pass

        index_score = sorted(score.items(), key=lambda (k, v): v, reverse=True)

        if index_score:
            top10_doc_index = [i[0] for i in index_score[:10]]
            return top10_doc_index
        else:
            print 'error occured' ,query_sent
            return -1, 0

In [4]:
def build_model_and_evaluate(model, query ,report=False):
    evaluate_row = []
    pred = model.predict(query)
    quest_index = 0
    for pred_index in pred:
        drow = dict()
        if report:
            print pred_index
        drow['question_ID'] = quest_index
        drow['prediction_ID'] = pred_index
        evaluate_row.append(drow)
        quest_index += 1
    return evaluate_row


# #build model for each document collaction

for document in dataset:
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, use_idf=True,norm='l1',stop_words=None, tokenizer=my_tokenizer)
    document_collections_sents = document['sentences']
    tfidf_matrix = tfidf_vectorizer.fit_transform(document_collections_sents)
    document['model'] = MostRelevantSentenceModel(vectorizer=tfidf_vectorizer,collection_matrix=tfidf_matrix)

#### Output with predtion and actual values

In [5]:
import csv
csv_file = open('evaluatin_test_results.csv', mode='w',)
fieldnames = ['document_ID', 'question_ID','question','prediction_ID','prediction_sentence']
writer = csv.DictWriter(csv_file, fieldnames=fieldnames, )
writer.writeheader()

ddi = 0
for document in dataset:
    questions = [i['question'] for i in document['qa']]
    model = document['model']
    result_row = build_model_and_evaluate(model, questions)
    doc_sents = document['sentences']
    for r in result_row:
        r['document_ID'] = ddi
        r['question'] = questions[r['question_ID']].encode('utf-8')
        r['prediction_sentence'] = doc_sents[r['prediction_ID'][0]].encode('utf-8')
        writer.writerow(r)
    ddi += 1
print 'Running time count:', time.time() - t0

error occured What was destroyed in the fire?
error occured What is the Barengraben?
error occured Who were the “intelligentia?” 
error occured What are phrynges?
error occured When was the shooting?
Running time count: 206.378999949


In [6]:
csv_file = 'evaluatin_test_results.csv'
with open(csv_file) as csvfile:
    readCSV = csv.DictReader(csvfile, delimiter=',')
    for row in readCSV:
        print(row['prediction_ID'])

[283, 282, 380, 353, 300, 39, 281, 292, 299, 311]
[283, 299, 282, 26, 380, 300, 155, 39, 281, 64]
[1, 282, 2, 15, 29, 90, 300, 79, 41, 164]
[2, 282, 55, 300, 41, 1, 298, 66, 67, 322]
[2, 282, 55, 78, 1, 29, 300, 41, 79, 130]
[5, 282, 114, 8, 300, 178, 295, 306, 298, 161]
[6, 282, 180, 300, 78, 2, 117, 298, 174, 66]
[7, 282, 300, 172, 213, 8, 181, 84, 6, 20]
[8, 282, 70, 300, 120, 100, 5, 7, 6, 109]
[282, 9, 113, 8, 313, 101, 380, 110, 27, 300]
[282, 380, 300, 283, 292, 299, 311, 353, 134, 298]
[244, 380, 282, 300, 10, 108, 175, 122, 251, 117]
[282, 270, 300, 11, 21, 298, 318, 8, 120, 203]
[282, 270, 248, 300, 12, 11, 122, 269, 161, 151]
[13, 282, 120, 300, 119, 315, 38, 53, 354, 258]
[244, 282, 251, 17, 300, 175, 212, 360, 229, 297]
[282, 283, 162, 212, 306, 17, 300, 366, 281, 169]
[282, 117, 292, 169, 300, 6, 242, 19, 145, 116]
[20, 282, 221, 252, 292, 169, 300, 321, 8, 35]
[175, 282, 283, 300, 366, 226, 17, 354, 162, 298]
[282, 269, 280, 300, 161, 126, 268, 204, 226, 166]
[282, 267, 

### Entity Extraction

In [7]:
def json_load_byteified(file_handle):
    return _byteify(json.load(file_handle, object_hook=_byteify),ignore_dicts=True)

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

with open("data/QA_test.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

import success


In [ ]:
import os
java_path = "C:/Program Files/Java" # replace this
os.environ['JAVAHOME'] = java_path

In [ ]:
!pip uninstall nltk
!pip install nltk

In [57]:
cwd = os.getcwd()
st = StanfordNERTagger(cwd+'\data\english.all.3class.distsim.crf.ser.gz',cwd+'\data\stanford-ner.jar')

if not os.path.isfile("NERtest.json"):    
    start = time.time()
    progressT = len(json_data)    
    listOfDocument=[]
    i=0
    for jd in json_data:
        aList=[]        
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['question']))) for x in jd['qa']]))
        #remove the below file if running on test set
        #aList.extend([st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['answer']))) for x in jd['qa']])])
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x))) for x in jd['sentences']]))
        listOfDocument.append(aList)
        i+=1
        sys.stdout.write('\r')
        sys.stdout.write("%d%%" % (i*100/progressT))
        sys.stdout.flush()    
    for document in range(0,len(listOfDocument)):
        #change [2] to [1] if test set
        for sentence in range(0,len(listOfDocument[document][1])):
            for word in range(0,len(listOfDocument[document][1][sentence])):   
                listOfDocument[document][1][sentence][word]= (listOfDocument[document][1][sentence][word][0],listOfDocument[document][1][sentence][word][1] if not listOfDocument[document][1][sentence][word][0].isdigit() else u'NUMBER')
    with open('NERtest.json', 'w') as outfile:
        json.dump(listOfDocument, outfile)
    end = time.time()
    print '\nTime spending:',end - start    
else:    
    print 'there is a file'
with open("NERtest.json") as json_file:
        json_dataNER = json_load_byteified(json_file)

100%
Time spending: 237.630000114


In [59]:
listOfDocument[0][0][0]

[(u'What', u'O'),
 (u'year', u'O'),
 (u'did', u'O'),
 (u'the', u'O'),
 (u'Crimean', u'O'),
 (u'War', u'O'),
 (u'begin', u'O')]

In [ ]:
for document in range(0,len(listOfDocument)):
        for sentence in range(0,len(listOfDocument[document])):
            for word in range(0,len(listOfDocument[document][2][sentence])):   
                listOfDocument[document][2][sentence][word]= (listOfDocument[document][2][sentence][word][0],listOfDocument[document][2][sentence][word][1] if not listOfDocument[document][2][sentence][word][0].isdigit() else u'NUMBER')